In [ ]:
#!unzip "/content/drive/MyDrive/Colab Notebooks/datasety.zip" -d "/content/drive/MyDrive/Colab Notebooks/datasety"

Archive:  /content/drive/MyDrive/Colab Notebooks/datasety.zip
  inflating: /content/drive/MyDrive/Colab Notebooks/datasety/README.dataset.txt  
  inflating: /content/drive/MyDrive/Colab Notebooks/datasety/README.roboflow.txt  
  inflating: /content/drive/MyDrive/Colab Notebooks/datasety/data.yaml  
   creating: /content/drive/MyDrive/Colab Notebooks/datasety/test/
   creating: /content/drive/MyDrive/Colab Notebooks/datasety/test/images/
 extracting: /content/drive/MyDrive/Colab Notebooks/datasety/test/images/cardboard_cardboard118_jpg.rf.11732f3af1118e4e659055cb9ec8190f.jpg  
 extracting: /content/drive/MyDrive/Colab Notebooks/datasety/test/images/cardboard_cardboard121_jpg.rf.acb33dd2e61c25675a2a68ca395bb2c5.jpg  
 extracting: /content/drive/MyDrive/Colab Notebooks/datasety/test/images/cardboard_cardboard125_jpg.rf.bac2214dc6e3a0cde423a1d72cd7c693.jpg  
 extracting: /content/drive/MyDrive/Colab Notebooks/datasety/test/images/cardboard_cardboard12_jpg.rf.a5aea259526ae4c370986b168416adf

For object Detection and cropping the image from that result we are training a yolo model (yolov8n) which is used for detecting objects

This model requires a good GPU support to train faster, on CPU it can crash

mounting Google drive

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


.




**Installing requirements **

In [7]:
!pip install gradio ultralytics Pillow
!pip install ultralytics roboflow

.


**GPU check**

In [3]:
import torch

print("✅ GPU available:", torch.cuda.is_available())

✅ GPU available: True


Training Yolo model with Epochs= 30, image input size= 416, Batch = 16 for avoiding GPU Crashes

In [5]:
from ultralytics import YOLO
import torch

# Confirm GPU
print("✅ GPU available:", torch.cuda.is_available())
print("🎯 Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

# Load a lightweight, efficient model
model = YOLO("yolov8n.pt")  # 'n' = nano, smallest and safest for free Colab GPUs

# Colab-safe and crash-resistant training settings
model.train(
    data="/content/drive/MyDrive/Colab Notebooks/datasety/data.yaml",
    epochs=30,               # More epochs, since training is lighter
    imgsz=416,               # Balanced image size (lower = faster, less memory)
    batch=16,                # ⚠️ Safer batch size for T4 GPUs (adjust if it crashes)
    workers=2,               # Light threading to reduce RAM load
    device=0,                # Use GPU
    freeze=0,                # Train all layers
    lr0=0.0015,              # Moderate learning rate
    patience=10,             # Early stopping for stability
    optimizer="Adam",        # Stable convergence
    cos_lr=True,             # Smooth LR decay
    mosaic=0.5,              # Moderate augmentation
    hsv_h=0.005, hsv_s=0.3, hsv_v=0.2,
    translate=0.05, scale=0.3, shear=0.5,  # Lighter transforms
    flipud=0.0, fliplr=0.3,
    mixup=0.0,               # Disable mixup to save memory
    name="custom_yolo_colab_safe"
)


✅ GPU available: True
🎯 Device: Tesla T4


100%|██████████| 6.25M/6.25M [00:00<00:00, 129MB/s]

Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/datasety/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.3, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.005, hsv_s=0.3, hsv_v=0.2, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0015, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.5, multi_scale=False, name=custom_yolo_colab_safe, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=10, perspectiv

  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384,

train: Scanning /content/drive/MyDrive/Colab Notebooks/datasety/train/labels.cache... 1238 images, 5 backgrounds, 0 corrupt: 100%|██████████| 1238/1238 [00:00<?, ?it/s]

...

                   all        363        362      0.847      0.781      0.868      0.725



30 epochs completed in 0.284 hours.
Optimizer stripped from runs/detect/custom_yolo_colab_safe/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/custom_yolo_colab_safe/weights/best.pt, 6.2MB

Validating runs/detect/custom_yolo_colab_safe/weights/best.pt...
Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:04<00:00,  2.89it/s]


                   all        363        362      0.846      0.781      0.868      0.726
             cardboard         85         85      0.987      0.892      0.984      0.936
                 glass        107        107      0.913      0.789      0.924      0.659
                 metal         64         64      0.896       0.75        0.9       0.74
                 paper         51         51      0.745      0.902      0.829      0.749
               plastic         37         37      0.778      0.855      0.898      0.762
                 trash         18         18      0.758        0.5      0.674      0.508
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/custom_yolo_colab_safe


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b5a714084d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

.


Loading and saving the model

In [ ]:
# Load model (from training run or pre-trained)
model = YOLO("/content/drive/MyDrive/Colab Notebooks/modelsave/best.pt")  # or any other path

# Save the model to a new location
model.save("/content/drive/MyDrive/Colab Notebooks/modelsave/best.pt")

.



making the final project from Yolov8n and EfficintnetV2S models
yolo will crop the picture and based in very big background but if yolo model becomes unable to detect the object it will pass the input to efficientnet model


In [13]:
import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

# --- Configuration for YOLOv8 Object Detection ---
# Define your 6 class names for YOLOv8 (must match your YOLOv8 training).
CLASS_NAMES_YOLO = ['paper', 'cardboard', 'plastic', 'metal', 'trash', 'glass']

# Path to your trained YOLOv8n model weights.
# IMPORTANT: This path has been updated as per your request.
# Ensure your 'best.pt' file is located at this exact path.
MODEL_PATH_YOLO = '/content/drive/MyDrive/Colab Notebooks/modelsave/best.pt'

# Padding in pixels to expand the cropped image dimensions.
PADDING_PIXELS = 20 # Example: Add 20 pixels of padding on each side

# --- Configuration for EfficientNetV2S Classification ---
# Path to your trained EfficientNetV2S Keras model.
# Ensure 'EfficientNetV2S.keras' is in the same directory as this script or provide its full path.
MODEL_PATH_CLASSIFICATION = '/content/drive/MyDrive/Colab Notebooks/modelsave/224EfficientNetV2S.keras'

# Define your class names for the classification model (must match your classification training).
CLASS_NAMES_CLASSIFICATION = ['Cardboard', 'Glass', 'Metal', 'Paper', 'Plastic', 'Trash']

# --- Load YOLOv8 Model ---
try:
    yolo_model = YOLO(MODEL_PATH_YOLO)
    print(f"YOLOv8 model loaded successfully from {MODEL_PATH_YOLO}")
except Exception as e:
    print(f"Error loading YOLOv8 model from {MODEL_PATH_YOLO}: {e}")
    print("Please ensure the YOLOv8 model path is correct and the model file exists.")
    exit()

# --- Load Classification Model ---
try:
    classification_model = tf.keras.models.load_model(MODEL_PATH_CLASSIFICATION)
    print(f"Classification model loaded successfully from {MODEL_PATH_CLASSIFICATION}")
except Exception as e:
    print(f"Error loading classification model from {MODEL_PATH_CLASSIFICATION}: {e}")
    print("Please ensure the Keras model path is correct and the model file exists.")
    exit()

# --- Classification Function for Cropped Images ---
def classify_cropped_image(img_np: np.ndarray) -> tuple[str, str]:
    """
    Classifies a single cropped image using the loaded EfficientNetV2S model.

    Args:
        img_np (np.ndarray): The input cropped image as a NumPy array.

    Returns:
        tuple[str, str]: A tuple containing:
            - str: A formatted string with the predicted class and confidence.
            - str: An HTML string for the final prediction status.
    """
    # Convert NumPy array to PIL Image
    # Ensure the image data type is uint8 before converting to PIL Image
    img_pil = Image.fromarray(img_np.astype('uint8'))

    # Resize and preprocess for the classification model
    img_pil = img_pil.resize((224, 224))
    img_array = np.array(img_pil, dtype=np.float32)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0) # Add batch dimension

    # Make prediction
    # verbose=0 suppresses the prediction progress bar output
    prediction = classification_model.predict(img_array, verbose=0)
    predicted_index = np.argmax(prediction)

    classification_result_str = ""
    final_prediction_status = ""

    # Ensure the predicted index is within the bounds of the defined class names
    if predicted_index < len(CLASS_NAMES_CLASSIFICATION):
        predicted_label = CLASS_NAMES_CLASSIFICATION[predicted_index]
        confidence = prediction[0][predicted_index]
        classification_result_str = f"Predicted: {predicted_label} (Conf: {confidence:.2f})"

        # Set latest prediction for status
        final_prediction_status = f"""
        <div style='text-align: center; font-size: 28px; font-weight: bold; color: #00FFAA; font-family: Arial, sans-serif;'>
            Prediction: {predicted_label}<br>
            <span style='font-size: 20px; color: #888;'>Confidence: {confidence:.2%}</span>
        </div>
        """
    else:
        classification_result_str = f"Prediction Error: Class index {predicted_index} out of bounds for classification model."
        final_prediction_status = f"""
        <div style='text-align: center; font-size: 20px; color: red; font-family: Arial, sans-serif;'>
            Prediction Error: Class index {predicted_index} out of bounds.
        </div>
        """
    return classification_result_str, final_prediction_status


# --- Combined Object Detection and Classification Function ---
def detect_and_classify_objects(image_np: np.ndarray) -> tuple[np.ndarray, np.ndarray | None, list, str]:
    """
    Performs object detection, crops detected objects, and then classifies each cropped object.
    If no objects are detected, it classifies the entire image.

    Args:
        image_np (np.ndarray): The input image frame as a NumPy array (H, W, C).

    Returns:
        tuple[np.ndarray, np.ndarray | None, list, str]: A tuple containing:
            - np.ndarray: The image frame with bounding boxes drawn on it (or the original if no detections).
            - np.ndarray | None: The first cropped image with its classification label (or None if no detections, or the full image if classified as a fallback).
            - list: A list of (cropped_image_np, classification_label_string) tuples for the gallery.
            - str: An HTML string representing the final prediction status.
    """
    annotated_frame = image_np.copy()
    first_cropped_image_output = None
    cropped_images_with_labels = []
    latest_prediction_status = ""

    # Perform YOLOv8 inference
    results = yolo_model(image_np, conf=0.5, iou=0.45, verbose=False)

    # Check if any objects were detected
    detections_found = False
    for r in results:
        if len(r.boxes) > 0:
            detections_found = True
            break

    if detections_found:
        for r in results:
            boxes = r.boxes  # Access detected bounding boxes
            for i, box in enumerate(boxes):
                x1_orig, y1_orig, x2_orig, y2_orig = map(int, box.xyxy[0])

                h, w, _ = image_np.shape
                x1_padded = max(0, x1_orig - PADDING_PIXELS)
                y1_padded = max(0, y1_orig - PADDING_PIXELS)
                x2_padded = min(w, x2_orig + PADDING_PIXELS)
                y2_padded = min(h, y2_orig + PADDING_PIXELS)

                color = (0, 255, 0)
                cv2.rectangle(annotated_frame, (x1_orig, y1_orig), (x2_orig, y2_orig), color, 2)

                if x2_padded > x1_padded and y2_padded > y1_padded:
                    cropped_object_np = image_np[y1_padded:y2_padded, x1_padded:x2_padded]

                    classification_result_str, current_prediction_status = classify_cropped_image(cropped_object_np)
                    latest_prediction_status = current_prediction_status

                    if i == 0: # Only assign the first detected and classified object
                        temp_img_for_single_output = cropped_object_np.copy()
                        (text_width, text_height), baseline = cv2.getTextSize(classification_result_str, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
                        rect_x1 = 0
                        rect_y1 = temp_img_for_single_output.shape[0] - text_height - baseline - 10
                        rect_x2 = text_width + 20
                        rect_y2 = temp_img_for_single_output.shape[0]
                        cv2.rectangle(temp_img_for_single_output, (rect_x1, rect_y1), (rect_x2, rect_y2), (0, 0, 0), -1)
                        cv2.putText(temp_img_for_single_output, classification_result_str, (10, temp_img_for_single_output.shape[0] - 10),
                                     cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                        first_cropped_image_output = temp_img_for_single_output

                    cropped_images_with_labels.append((cropped_object_np, classification_result_str))
    else:
        # If no objects detected by YOLO, classify the entire image
        print("No objects detected by YOLO. Classifying the entire image.")
        classification_result_str, latest_prediction_status = classify_cropped_image(image_np)

        # For the "First Cropped Object" output, display the full classified image
        first_cropped_image_output = image_np.copy()
        (text_width, text_height), baseline = cv2.getTextSize(classification_result_str, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        rect_x1 = 0
        rect_y1 = first_cropped_image_output.shape[0] - text_height - baseline - 10
        rect_x2 = text_width + 20
        rect_y2 = first_cropped_image_output.shape[0]
        cv2.rectangle(first_cropped_image_output, (rect_x1, rect_y1), (rect_x2, rect_y2), (0, 0, 0), -1)
        cv2.putText(first_cropped_image_output, classification_result_str, (10, first_cropped_image_output.shape[0] - 10),
                     cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Add the full image with its classification to the gallery as well
        cropped_images_with_labels.append((image_np, "Full Image: " + classification_result_str))

    return annotated_frame, first_cropped_image_output, cropped_images_with_labels, latest_prediction_status


# --- Gradio Interface Setup ---
iface = gr.Interface(
    fn=detect_and_classify_objects,
    inputs=gr.Image(type="numpy", label="Webcam Feed"),
    outputs=[
        gr.Image(type="numpy", label="Detected Objects (Full Frame)"),
        gr.Image(type="numpy", label="First Cropped Object (Classified)", height=250, width=250),
        gr.Gallery(label="All Cropped Detections with Classification", columns=1, rows=1, object_fit="contain", height="auto"),
        gr.HTML(label="Latest Prediction Status")
    ],
    live=True,
    title="YOLOv8 Object Detection & EfficientNetV2S Classification",
    description="This application performs real-time object detection using YOLOv8, "
                "crops the detected objects, and then classifies each cropped object "
                "using an EfficientNetV2S model. If no objects are detected by YOLO, "
                "the entire frame will be classified by the EfficientNetV2S model. "
                f"YOLOv8 is configured to detect: {', '.join(CLASS_NAMES_YOLO)}. "
                f"The classification model predicts categories: {', '.join(CLASS_NAMES_CLASSIFICATION)}."
)

# --- Launch Gradio Interface ---
if __name__ == "__main__":
    print("Launching Gradio interface...")
    iface.launch()
    print("Gradio interface launched. Open the public URL in your browser.")

YOLOv8 model loaded successfully from /content/drive/MyDrive/Colab Notebooks/modelsave/best.pt
Classification model loaded successfully from /content/drive/MyDrive/Colab Notebooks/modelsave/224EfficientNetV2S.keras
Launching Gradio interface...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://60a5b8589ca4526c2b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Gradio interface launched. Open the public URL in your browser.
